## 주제

운수종사자 인지적 특성 데이터를 활용한 교통사고 위험 예측 AI 모델 개발

## 데이터

### 주요 데이터 구성

- baseline_submit.zip: 베이스라인 코드 기반 제출 대회 예시 파일
- data/sample_submission.csv: 모델 출력 결과 형식 예시 CSV
    - Test_id: 검사 식별 ID
    - Label: 해당 검사 결과로부터 예측한 교통사고 위험군에 속할 확률(0~1)
- data/train.csv: 학습 샘플 목록이 포함된 CSV
    - Test_id: 검사 식별 ID
    - Test: 검사 종류(A: 신규 자격 검사, B: 자격 유지 검사)
    - Label: 해당 검사 결과가 교통사고 위험군에 해당 하는 지의 여부(0: 교통사고 위험군 X, 1: 교통사고 위험군 O)
- data/test.csv: 테스트 샘플 목록이 포함된 CSV
    - Test_id: 검사 식별 ID
    - Test: 검사 종류(A: 신규 자격 검사, B: 자격 유지 검사)
- data/train/A.csv: 학습 샘플 중 신규 자격 검사 샘플
    - Test_id: 검사 식별 ID
    - Test: 검사 종류(A: 신규 자격 검사, B: 자격 유지 검사)
    - PrimaryKep: 운수종사자 식별 ID
    - Age: 검사 시점의 운수종사자 나이(a: 0~4, b: 5~9, ex) 30a: 30~34, 30b: 35~59)
    - TestDate: 검사 시점(년월: yyyymm)
- data/train/B.csv: 학습 샘플 중 자격 유지 검사 샘플
    - Test_id: 검사 식별 ID
    - Test: 검사 종류(A: 신규 자격 검사, B: 자격 유지 검사)
    - PrimaryKep: 운수종사자 식별 ID
    - Age: 검사 시점의 운수종사자 나이(a: 0~4, b: 5~9, ex) 30a: 30~34, 30b: 35~59)
    - TestDate: 검사 시점(년월: yyyymm)
- data/test/A.csv: 테스트 샘플 중 신규 자격 검사 샘플
- data/test/B.csv: 테스트 샘플 중 자격 유지 검사 샘플

### AutoML 사용

```python
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label='target', eval_metric='...').fit(train_data, time_limit=..., presets='best_quality')
```

## 코드 흐름

1. 데이터 로드 및 기본 전처리
    - 훈련/검증/테스트 데이터셋 불러옴
    - 결측치 처리, 타입 변환 등 수행
2. 피처 엔지니어링
    - 이전 시점의 라벨(검사 A, 검사 B 각각)과 이전 라벨별 개수를 주요 피처로 활용
    - TestDate가 유의미한 피처임
    - 통계량(평균, 표준편차, z-score)도 시도했지만, 이전 라벨 활용이 더 효과적이었음
3. 모델 학습 및 자동화 도구 사용
    - AutoGluon 같은 자동 머신러닝(AutoML) 도구 사용
    - 메모리 부족 등의 이유로 피처 타입 변경, 간소화
4. 검증 및 제출
    - 검증 지표로는 A, B 각각의 예측 점수 사용
    - 테스트 데이터에 대해 예측하고 제출 파일 생성
5. 추가 전략 시도
    - TestDate 외 변수들로 학습시키고 앙상블 시켜보는 전략 시도

## 차별점 및 배울점

- 도메인에 맞는 피처 엔지니어링의 중요성
    - 이전 검사 결과와 이전 라벨의 누적 개수같은 시계열적/행동적 특징을 피처로 만들었음
- TestDate 피처의 유의미함 확인
    - 도메인 특성과 시간 추세가 중요하다는 점 확인
- AutoML을 활용한 빠른 베이스라인 확보
    - AutoGluon 같은 AutoML 도구만으로도 높은 점수를 얻었음
    - 초기 실험 단계에서 AutoML을 적극 활용 후 성능 최적화 단계에서 커스텀 피처 엔지니어링을 추가하는 접근이 효율적임
- 메모리 최적화
    - 메모리가 부족 문제를 피처의 최대, 최소 범위를 확인하여 int8, int16등으로 바꾸어 해결
- 검증 전략과 모델 해석의 중요성
    - LightGBM에 L1 규제를 적용했더니 변수 중요도가 0이 되는 문제 발생
    - 모델이 피처를 어떻게 해석하는지 확인해야 하며, 파라미터 조절이 피처의 정보량에도 영향을 미친다는 것을 알게됨